In [16]:
import tweepy
import json
import psycopg2
import data_cred as cred
from geopy.geocoders import Nominatim

In [17]:
consumer_key = cred.api_key
consumer_secret = cred.api_secrect_key
access_token = cred.access_token
access_token_secret = cred.access_token_secrest
geolocator = Nominatim(user_agent="twitter_scrapper")

In [18]:
def api_connection():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
        print("auth ok")
    except Exception as e:
        print(e.Exception)

    return api

In [19]:
api = api_connection()

auth ok


In [20]:
def extract_tweets(hashtag, count, geo):
    #api = api_connection()
    print('Init..')
    tweets_json = []
    tuits_cursor = tweepy.Cursor(api.search,q=hashtag, lang='es', result_type='recent', rpp=100, include_rts=False,tweet_mode='extended',geocode=geo).items(count)
    #a = tuits_cursor.next()
    for tweet in tuits_cursor:
        if (not tweet.retweeted and 'RT @' not in tweet.full_text):
            tweets_json.append(tweet._json)
            #avatar, geo_loc, verificada?, tag, username, user_id, entities(img, url, etc), source(android,ios,web)
    print('Ending..')
    return tweets_json

In [21]:
def apply_filter(topic):
    since_until=' since:2020-10-14 until:2020-10-16'
    filter_retweets=" -filter:retweets"
    topic = topic+since_until
    topic = topic+filter_retweets
    return topic

In [22]:
def connect_db(user,password,host,port,database):
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")              
    return cursor,connection

In [23]:
def search_by_geo(geo_data, topic, count, cursor, connection ):
    for geo in geo_data:
        location = geolocator.reverse(geo[0])
        geo_t = geo[0]+',400km'
        tweets = extract_tweets(apply_filter(topico_buscar_a),count, geo_t)
        print(location.raw['address']['city'],location.raw['address']['country'], f' topic = {topic},  Count = {len(tweets)}')
        insert_tweets(tweets, topic,cursor,connection,location )
    return tweets


In [24]:
def insert_tweets(tweets, topic,cursor,connection,location):
    for  tweet in tweets:
        #json.dumps(x)
        script = "INSERT INTO data_tweets (ID, tweet , topic, city_searched, country_searched) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (ID) DO NOTHING;;"
        cursor.execute(script,(tweet['id'],json.dumps(tweet),topic, location.raw['address']['city'], location.raw['address']['country'] ))
        connection.commit()


In [25]:
location = geolocator.reverse('-12.0448728,-77.0488376')
location.raw['address']['city'],location.raw['address']['country']

('Lima', 'Peru')

In [26]:
cursor,connection = connect_db(cred.database_user,cred.database_pass,"192.168.0.22",'5432','data_tweets')

You are connected to -  ('PostgreSQL 12.4 (Ubuntu 12.4-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-10ubuntu2) 9.3.0, 64-bit',) 



In [27]:
geo_data = cred.geo_data
topico_buscar_a ='trump'
topico_buscar_b ='biden'
count = 8000
print(geo_data)

[['-33.4672832,-70.652568', 'santiago, chile'], ['-34.6025287,-58.3796486', ' buenos aires, argentina'], ['-23.5861315,-46.6582803', ' sao paulo, brasil'], ['-34.8207362,-56.3765221', 'montevideo, uruguay'], ['-31.3992876,-64.2643839', 'cordoba, argentina'], ['-32.8832979,-68.8760287', ' mendoza, argentina'], ['-31.6145747,-60.6980372', ' santa fe, argentina'], ['7.7721289,-72.226306', ' San cristobal , venezuela'], ['10.2241386,-67.3411383', ' La victoria, Aragua , venezuela'], ['10.4686988,-67.0304528', 'caracas, venezuela'], ['4.657911,-74.0959651', 'Bogota, colombia'], ['-12.0448728,-77.0488376', 'Lima, Peru']]


In [13]:
tweets_a= search_by_geo(geo_data, topico_buscar_a, count, cursor, connection  )

Init..
Ending..
Santiago Chile  topic = trump,  Count = 377
Init..
Ending..
Buenos Aires Argentina  topic = trump,  Count = 892
Init..
Ending..
São Paulo Brasil  topic = trump,  Count = 85
Init..
Ending..
Montevideo Uruguay  topic = trump,  Count = 844
Init..
Ending..
Córdoba Argentina  topic = trump,  Count = 277
Init..
Rate limit reached. Sleeping for: 900


TweepError: Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [28]:
tweets_b= search_by_geo(geo_data, topico_buscar_b, count, cursor, connection  )


Init..
Ending..
Santiago Chile  topic = biden,  Count = 380
Init..
Ending..
Buenos Aires Argentina  topic = biden,  Count = 900
Init..
Ending..
São Paulo Brasil  topic = biden,  Count = 86
Init..
Rate limit reached. Sleeping for: 948
Ending..
Montevideo Uruguay  topic = biden,  Count = 853
Init..
Ending..
Córdoba Argentina  topic = biden,  Count = 283
Init..
Ending..
Ciudad de Mendoza Argentina  topic = biden,  Count = 476
Init..
Ending..
Santa Fe Argentina  topic = biden,  Count = 763
Init..
Rate limit reached. Sleeping for: 903
Ending..
San Cristóbal Venezuela  topic = biden,  Count = 1991
Init..
Rate limit reached. Sleeping for: 919


TweepError: Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))